# **Install and Import Neccessary Libraries**

In [ ]:
#Install Neccessary Libraries
!pip install flask
!pip install pandas
!pip install --upgrade google-cloud-bigquery
!pip install flask-ngrok

In [10]:
#Make Neccessary Imports
from flask import Flask, render_template
import pandas as pd
from google.cloud import bigquery
from flask_ngrok import run_with_ngrok

In [11]:
# Construct a BigQuery client object.
client = bigquery.Client()

In [ ]:
app = Flask(__name__)

@app.route("/")
def transactions():
    # Connect to BigQuery
    client = bigquery.Client()
    dataset_id = "bigquery-public-data"
    table_id = "bitcoin_blockchain.transactions"
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)

    # Query the data
    query = """
    SELECT *
    FROM (
        SELECT transaction_id, COUNT(transaction_id) AS dup_transaction_count
        FROM `bigquery-public-data.bitcoin_blockchain.transactions`
        GROUP BY transaction_id
    )
    WHERE dup_transaction_count > 1;
    """
    try:
        df = client.query(query).to_dataframe()
    except Exception as e:
        return str(e)

    # Return the data to the template
    return render_template("transactions.html", data=df.to_html())

if __name__ == "__main__":
    app.run()
